In [1]:
import sys, os
import pybullet as p
import time
from math import *
import json

import modern_robotics as mr

from Indy7  import *
#--------------------------------------------
## Env Setup
CONTROL_FREQ = 1000.0
endTime = 20;
dt = 1/CONTROL_FREQ;
urdf_name = "model/indy7.urdf"
indy7 = Indy7(p,urdf_name = urdf_name, CONTROL_FREQ=CONTROL_FREQ)
indy7.resetJointStates(np.array([ 0.   ,  0,-np.pi/2.0,0,-np.pi/2.0,0]).T);
p.loadURDF("model/plane.urdf")
p.loadURDF("model/axis.urdf")
qddot = np.zeros([6,1])
g = np.array([0,0,-9.8])
Ftip = [0,0,0,0,0,0]

# READ MR_INFO.JSON
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data
    
json_file_path = 'MR_info.json'
data_from_json = read_json_file(json_file_path)


M=data_from_json['M']
Slist = np.array(data_from_json['baseSlist'])
Blist = np.array(data_from_json['baseBlist'])
Mlist_ = np.array(data_from_json['Mlist'])
Mlist = np.zeros([4,4,7]);
Glist = np.zeros([6,6,6]);
Glist=[]
Mlist=[]
for i in range(Mlist_.shape[0]):
	Mlist.append(Mlist_[i,0:4,0:4]);
Glist_ = np.array(data_from_json['Glist'])

for i in range(Glist_.shape[0]):
	Glist.append(Glist_[i,0:6,0:6]);
	
	
	

for t in np.arange(0,endTime,dt):
	q,qdot = indy7.getJointStates();
	T = FKinSpace(M,Slist,q);
	Js,Jb,Ja,pinvJs,pinvJb, pinvJa = indy7.getJacobian(M,Slist,Blist,q);
	tau_grav =mr.InverseDynamics(q, qdot, qddot, g, Ftip, Mlist,  Glist, Slist,0);	
	indy7.setTorques(tau_grav)
	indy7.drawEEF();
	indy7.step()
	time.sleep(dt)
exit(0)


============================JOINT_INFO====================================
Joint Num :  0 	 Joint Name :  b'world_joint' 	 Joint Type :  JOINT_FIXED
Joint Num :  1 	 Joint Name :  b'joint0' 	 Joint Type :  JOINT_REVOLUTE
Joint Num :  2 	 Joint Name :  b'joint1' 	 Joint Type :  JOINT_REVOLUTE
Joint Num :  3 	 Joint Name :  b'joint2' 	 Joint Type :  JOINT_REVOLUTE
Joint Num :  4 	 Joint Name :  b'joint3' 	 Joint Type :  JOINT_REVOLUTE
Joint Num :  5 	 Joint Name :  b'joint4' 	 Joint Type :  JOINT_REVOLUTE
Joint Num :  6 	 Joint Name :  b'joint5' 	 Joint Type :  JOINT_REVOLUTE
Joint Num :  7 	 Joint Name :  b'tcp_joint' 	 Joint Type :  JOINT_FIXED
[b'joint0', b'joint1', b'joint2', b'joint3', b'joint4', b'joint5']


: 